#WEB Scraping


We would like to find all CSUF Faculty and Administration from 
CSUF 2020-2021 University Catalog at
   https://catalog.fullerton.edu/content.php?catoid=61&navoid=7316
using web scraping with Python. 

For each faculty or staff, we want to get the following information:

- Name: Faculty or Staff name

- Year: the year of this faculty or staff joining CSUF
    
- Position: Position of this faculty or staff at CSUF
    
- Education: Degrees of this faculty or staff
    
For example, for our Preseident at CSUF, 

   - Name = "Virjee, Framroze M." 

   - Year = 2018

   - Position = "President"

   - Education = "B.A. University of California, Santa Barbara; J.D. University of California, Hastings College of the Law"

Please note that Year for each Faculty or Staff should be an integer 
number.




Please follow the following instructions to complete all tasks.

**Task 1. Use web scraping with Python to extract the information 
of ``Name``, ``Year``, ``Position`` and ``Education`` for each faculty 
or staff. We use a list ``Names`` to record all names of all Faculty and Staff.  The ith element 
of the list ``Names`` is
the name of the ith Faculty or Staff. The meaning of other lists ``Years``, 
``Positions`` and ``Educations`` are similar. If this information is missing, 
assign the value "N/A". For example, there is no education information for
the person "Brown, Alisha", the value of ``Educations`` corrspodning to 
this person is "N/A".** 

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import Request, urlopen

url = 'https://catalog.fullerton.edu/content.php?catoid=61&navoid=7316'



req = Request(url , headers={'User-Agent': 'Chrome/70.0.3538.77'})

page = urlopen(req).read()
soup = BeautifulSoup(page, "html.parser")

**Task 2.  Merge the lists ``Names``, ``Years``,  ``Positions`` and ``Educations`` into a pandas 
DataFrame ``FacultyStaff2020to2021``. 
Put the list ``Names`` in the first column
of the DataFrame ``FacultyStaff2020to2021``, 
the list ``Years`` in the second column,
the list ``Positions`` in the third column,
and the list ``Educations`` in the fourth column.
Use Name, Year, Position, and Education for the 
column names of the DataFrame ``FacultyStaff2020to2021``. 
Then display the first five row of this DataFrame.**

<img src="https://drive.google.com/uc?export=view&id=1KJedbcW2zF0kfzbtSmNYfZQyZTMnsDBO" width="750"></img>

In [2]:
## TYPE YOUR CODE HERE
####### 1.  FINAL Code for getting name and year list

strong_AZ_tags = soup.select('p strong a')

strong_tags = soup.select('p strong')


pattern = re.compile(r"[A-Z]")

name_year_list = list()
for name_year in strong_tags:
    if(name_year is not None):
        name_year.get_text().strip()
        if(len(name_year.get_text())<2 and name_year.get_text().isalpha() ):
            continue
        else:
            name_year_list.append(name_year.get_text().strip())
    else:
        continue
#print(name_year_list[0:40])



 ####################   1.1 replace '(' ')' AND ',' WITH SPACE
final_list = list()

for x in range(len(name_year_list)):
    #print(name_year_list[x])
    name_year_list[x] = name_year_list[x].replace("(",'')
    name_year_list[x] = name_year_list[x].replace(")",'')
    name_year_list[x] = name_year_list[x].replace(",",'')
#print(name_year_list[0:40])


####### 2 FINAL extracting years for others
temp_year_list = list()

for year in name_year_list:
    if(year.isnumeric()):
        temp_year_list.append(year)
    else:
        date = ""
        for i in range(len(year)):
            if(year[i].isnumeric()):
                date+=year[i]
        temp_year_list.append(date)


####    remove spaces and commas from year list

year_list = list()
for ele in temp_year_list:
    if ele.isnumeric():
        year_list.append(ele)
    else:
        continue


###### 2.2
#print(len(name_year_list))
for q in name_year_list:
    if(len(q)<2):
        name_year_list.remove(q)

#print(len(name_year_list))


##### 3 FINAL  Extract name
temp_name_year_list = name_year_list
temp = list()

for element in range(len(temp_name_year_list)):
    if temp_name_year_list[element].isnumeric():
        name_case1=''
        #print("####",temp_name_year_list[element])
        if(temp_name_year_list[element-2].isnumeric() or temp_name_year_list[element-2][-1].isnumeric()):
            print()
        else:
            name_case1 += temp_name_year_list[element-2]
            #print("element1",element)
            #print(name_case1)

        name_case1+=" "
        if(temp_name_year_list[element-1].isnumeric() or temp_name_year_list[element-1][-1].isnumeric()):
            print()
        else:
            name_case1 += temp_name_year_list[element-1]   
            #print(name_case1)
            #print("element2",element)
        temp.append(name_case1)

    else:
        name_case2=''
        if temp_name_year_list[element][-1].isnumeric():
            #print(temp_name_year_list[element])
            for i in temp_name_year_list[element]:
                #print("element3",temp_name_year_list[element])
                if(i.isnumeric()):
                    continue
                else:
                    name_case2+=i
            temp.append(name_case2)
            
#print(temp)                # No of people 1287
#print(len(temp))

name_list = list()
name_list = temp

######### Version 2 FINAL Positions 
p_tags = soup.select('p')
positions_list = list()

for p in p_tags:
    if p.em is not None:
        dat = p.get_text()
        d_list = dat.split('\n\n')
  
        for element in d_list:
            pos = element.split('\r\n')[0].split(')')[-1][1:].strip()
            positions_list.append(pos)

    elif p.em is None and p.strong is not None and p.a is None:
          pos_edu = p.text
          pos = pos_edu.split(')')[1][2:].split(';')[0].split('\r')[0]
          
          positions_list.append(pos)

############################## FINAL Education first few educations
edu_list = list()

for strong in soup('strong'):
    strong.decompose()

for em in soup("em"):
    em.decompose()

# for br in soup('br'):
#     br.decompose()

p_tags_without_other_tags = soup.find_all('p')

for x in p_tags_without_other_tags:
    #print(x.get_text())
    edu_list.append(x.get_text())

temp_edu = list()
for line in edu_list:
    #print(line)
    line = line.replace(u'\xa0',u' ')
    line = line.replace(u'\r\n',u' ')
    line = line.strip()
    if(line is ''):
        continue
    else:
        temp_edu.append(line)

temp_edu = temp_edu[6:12]
## Education of after first few by removing br tag

for strong in soup('strong'):
    strong.decompose()

for em in soup("em"):
    em.decompose()
    
for br in soup('br'):
    br.decompose()

p_tags_without_br_tags = soup.find_all('p')


t_l = list()
i = 0
for x in p_tags_without_br_tags:
    if(i<6):
        i+=1
        x.decompose()  
    else:
        #print(x.get_text())        
        t_l.append(x.get_text())

t_l = list(t_l)

for line in t_l:
    #print(line)
    line = line.replace(u'\xa0',u' ')
    line = line.replace(u'\r\n',u' ')
    line = line.strip()
    if(line is None):
        temp_edu.append('NA')
        continue
    else:
        temp_edu.append(line)


new_list1 = list()
new_list2 = list()
education_list = list()

new_list1 = temp_edu[:6]
#print(new_list1)

new_list2 = temp_edu[13:]
#print(new_list2)

education_list = new_list1 + new_list2
#print(len(education_list))

## Clean education
for item in education_list:
    item = item.replace(',',' ')
    item = item.strip()

#convert year list to integers
for i in range(0, len(year_list)):
    year_list[i] = int(year_list[i])

###### FINAL LIST
Names = name_list
Years = year_list
Positions = positions_list
Educations = education_list


# print(len(Names))
# print(len(Years))
# print(len(Positions))
# print(len(Educations))

##### Converting to Dataframe
FacultyStaff2020to2021 = pd.DataFrame(
    {'Name': name_list[:1272],
     'Year': year_list[:1272],
     'Position': positions_list[:1272],
     'Education': education_list[:1272]
    })


In [3]:
## Do not change the following code
print(Names[:5])
print(Years[:5])
print(Positions[:5])
print(Educations[:5])

['Virjee Framroze M. ', 'Abdoli Mansour ', 'Abdou Noha ', 'Abedi Vahideh S. ', 'Abnet Dustin A. ']
[2018, 2017, 2018, 2013, 2014]
['President', 'Lecturer in Mathematics', 'Associate Director, Institutional Research', 'Associate Professor of Information Systems and Decision Sciences', 'Assistant Professor of American Studies']
['B.A. University of California, Santa Barbara; J.D. University of California, Hastings College of the Law', 'M.S. California State University, Fullerton', 'B.A. American University in Cairo; M.P.A. California State Polytechnic University, Pomona; Ed.D. California State University, Fullerton', 'Ph.D. University of Toronto, School of Graduate Studies', 'B.A., M.A. Miami University; Ph.D. Indiana University']
